In [1]:
install.packages("dplyr")
install.packages("SPEI")
install.packages("zoo")
install.packages("arrow")

Installing package into 'C:/Users/Reslan Al Tinawi/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Reslan Al Tinawi\AppData\Local\Temp\Rtmp2TF9iG\downloaded_packages


Installing package into 'C:/Users/Reslan Al Tinawi/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'SPEI' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Reslan Al Tinawi\AppData\Local\Temp\Rtmp2TF9iG\downloaded_packages


Installing package into 'C:/Users/Reslan Al Tinawi/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'zoo' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Reslan Al Tinawi\AppData\Local\Temp\Rtmp2TF9iG\downloaded_packages


Installing package into 'C:/Users/Reslan Al Tinawi/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



package 'arrow' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Reslan Al Tinawi\AppData\Local\Temp\Rtmp2TF9iG\downloaded_packages


In [2]:
library(dplyr)
library(SPEI)
library(zoo)
library(fs)
library(arrow)

Warning message:
"package 'dplyr' was built under R version 4.3.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'SPEI' was built under R version 4.3.3"
# Package SPEI (1.8.1) loaded [try SPEINews()].

Warning message:
"package 'zoo' was built under R version 4.3.3"

Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Warning message:
"package 'fs' was built under R version 4.3.3"
Warning message:
"package 'arrow' was built under R version 4.3.3"

Attaching package: 'arrow'


The following object is masked from 'package:utils':

    timestamp




# Load the data:

In [3]:
terra_climate_data_path <- path("..", "data", "raw", "terra_climate")

In [4]:
data <- read.csv(terra_climate_data_path / "terra_climate_monthly_data.csv")

In [5]:
head(data)

,system.index,actual_evapotranspiration,community,date,max_temperature,min_temperature,potential_evapotranspiration,precipitation,province,soil_moisture,.geo
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,199801_00000000000000000003,270.4608,Andalucía,1998-01-01T00:00:00,12.18580,3.825242,370.2276,25.94601,Almería,592.070,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,199801_0000000000000000000a,413.3929,Andalucía,1998-01-01T00:00:00,15.15256,8.217208,413.4022,67.28744,Cádiz,1558.161,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,199801_0000000000000000000d,356.7229,Andalucía,1998-01-01T00:00:00,12.99764,4.959517,356.7316,49.50801,Córdoba,1434.690,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,199801_00000000000000000011,307.9098,Andalucía,1998-01-01T00:00:00,10.35408,1.802307,328.9167,35.85276,Granada,1054.164,"{""type"":""MultiPoint"",""coordinates"":[]}"
5,199801_00000000000000000014,370.8258,Andalucía,1998-01-01T00:00:00,14.14617,7.450944,370.8258,56.01782,Huelva,1569.069,"{""type"":""MultiPoint"",""coordinates"":[]}"
6,199801_00000000000000000016,329.6043,Andalucía,1998-01-01T00:00:00,11.45454,3.626087,348.0010,33.86239,Jaén,897.465,"{""type"":""MultiPoint"",""coordinates"":[]}"


# Preprocess the data:

Drop dummy columns:

In [6]:
data <- subset(data, select = -c(system.index, .geo))

Convert column `date` to `Date` type:

In [7]:
data$date <- as.Date(data$date, format = "%Y-%m-%d")

Sort by date:

In [8]:
data <- data %>% arrange(province, date)

Drop columns for no associated province:

In [9]:
data <- data %>% filter(province != 'Territorios no asociados a ninguna provincia')

In [10]:
head(data)

,actual_evapotranspiration,community,date,max_temperature,min_temperature,potential_evapotranspiration,precipitation,province,soil_moisture
,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,440.8385,Galicia,1998-01-01,12.49135,7.503345,440.8385,133.47992,A Coruña,2096.876
2,383.3361,Galicia,1998-02-01,14.84645,6.652516,389.5783,54.18859,A Coruña,2086.150
3,609.0688,Galicia,1998-03-01,16.90857,8.113772,744.0929,39.99172,A Coruña,1856.838
4,669.5078,Galicia,1998-04-01,13.28331,8.304465,669.5078,253.09233,A Coruña,2096.483
5,963.9466,Galicia,1998-05-01,17.22014,11.028989,1011.3938,96.79639,A Coruña,2015.058
6,857.4879,Galicia,1998-06-01,20.85127,12.991255,1115.9988,46.65924,A Coruña,1600.850


Add `water_balance` column which accounts for precipitation minus potential evapotranspiration:

In [11]:
data <- data %>% mutate(water_balance = precipitation - potential_evapotranspiration)

In [12]:
head(data)

,actual_evapotranspiration,community,date,max_temperature,min_temperature,potential_evapotranspiration,precipitation,province,soil_moisture,water_balance
,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,440.8385,Galicia,1998-01-01,12.49135,7.503345,440.8385,133.47992,A Coruña,2096.876,-307.3586
2,383.3361,Galicia,1998-02-01,14.84645,6.652516,389.5783,54.18859,A Coruña,2086.150,-335.3897
3,609.0688,Galicia,1998-03-01,16.90857,8.113772,744.0929,39.99172,A Coruña,1856.838,-704.1012
4,669.5078,Galicia,1998-04-01,13.28331,8.304465,669.5078,253.09233,A Coruña,2096.483,-416.4154
5,963.9466,Galicia,1998-05-01,17.22014,11.028989,1011.3938,96.79639,A Coruña,2015.058,-914.5974
6,857.4879,Galicia,1998-06-01,20.85127,12.991255,1115.9988,46.65924,A Coruña,1600.850,-1069.3396


In [13]:
calculate_spei <- function(df, scales = c(1, 3, 6, 9, 12, 24)) {
  water_balance <- df$precipitation - df$potential_evapotranspiration
  water_balance_ts <- zoo(water_balance, order.by = df$date)
  start_year <- as.numeric(format(min(df$date), "%Y"))
  start_month <- as.numeric(format(min(df$date), "%m"))
  water_balance_ts <- ts(coredata(water_balance_ts), start = c(start_year, start_month), frequency = 12)
  
  spei_values <- lapply(scales, function(scale) {
    spei(water_balance_ts, scale = scale)$fitted
  })
  names(spei_values) <- paste0("SPEI_", scales, "M")
  
  return(as.data.frame(spei_values))
}

In [14]:
calculate_spi <- function(df, scales = c(1, 3, 6, 9, 12, 24)) {
  precipitation_ts <- zoo(df$precipitation, order.by = df$date)
  start_year <- as.numeric(format(min(df$date), "%Y"))
  start_month <- as.numeric(format(min(df$date), "%m"))
  precipitation_ts <- ts(coredata(precipitation_ts), start = c(start_year, start_month), frequency = 12)
  
  spi_values <- lapply(scales, function(scale) {
    spi(precipitation_ts, scale = scale)$fitted
  })
  names(spi_values) <- paste0("SPI_", scales, "M")
  
  return(as.data.frame(spi_values))
}

In [15]:
data <- data %>%
  group_by(province) %>%
  do({
    spei_vals <- calculate_spei(.)
    spi_vals <- calculate_spi(.)
    cbind(., spei_vals, spi_vals)
  })

[1] "Calculating the Standardized Precipitation Evapotranspiration Index (SPEI) at a time scale of 1. Using kernel type 'rectangular', with 0 shift. Fitting the data to a log-Logistic distribution. Using the ub-pwm parameter fitting method. Checking for missing values (`NA`): all the data must be complete. Using the whole time series as reference period. Input type is tsvector. Time series spanning Jan 1998 to Dec 2023, with frequency = 12."
[1] "Calculating the Standardized Precipitation Evapotranspiration Index (SPEI) at a time scale of 3. Using kernel type 'rectangular', with 0 shift. Fitting the data to a log-Logistic distribution. Using the ub-pwm parameter fitting method. Checking for missing values (`NA`): all the data must be complete. Using the whole time series as reference period. Input type is tsvector. Time series spanning Jan 1998 to Dec 2023, with frequency = 12."
[1] "Calculating the Standardized Precipitation Evapotranspiration Index (SPEI) at a time scale of 6. Using 

In [16]:
head(data)

actual_evapotranspiration,community,date,max_temperature,min_temperature,potential_evapotranspiration,precipitation,province,soil_moisture,water_balance,⋯,SPEI_6M,SPEI_9M,SPEI_12M,SPEI_24M,SPI_1M,SPI_3M,SPI_6M,SPI_9M,SPI_12M,SPI_24M
<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
440.8385,Galicia,1998-01-01,12.49135,7.503345,440.8385,133.47992,A Coruña,2096.876,-307.3586,⋯,NA,NA,NA,NA,-0.1545215,NA,NA,NA,NA,NA
383.3361,Galicia,1998-02-01,14.84645,6.652516,389.5783,54.18859,A Coruña,2086.150,-335.3897,⋯,NA,NA,NA,NA,-0.7987331,NA,NA,NA,NA,NA
609.0688,Galicia,1998-03-01,16.90857,8.113772,744.0929,39.99172,A Coruña,1856.838,-704.1012,⋯,NA,NA,NA,NA,-1.2114096,-1.00419701,NA,NA,NA,NA
669.5078,Galicia,1998-04-01,13.28331,8.304465,669.5078,253.09233,A Coruña,2096.483,-416.4154,⋯,NA,NA,NA,NA,2.0199092,-0.05369823,NA,NA,NA,NA
963.9466,Galicia,1998-05-01,17.22014,11.028989,1011.3938,96.79639,A Coruña,2015.058,-914.5974,⋯,NA,NA,NA,NA,0.6237760,0.60697137,NA,NA,NA,NA
857.4879,Galicia,1998-06-01,20.85127,12.991255,1115.9988,46.65924,A Coruña,1600.850,-1069.3396,⋯,0.3709908,NA,NA,NA,-0.2398450,1.72808735,-0.1502317,NA,NA,NA


In [17]:
colSums(is.na(data))

actual_evapotranspiration                    community 
                           0                            0 
                        date              max_temperature 
                           0                            0 
             min_temperature potential_evapotranspiration 
                           0                            0 
               precipitation                     province 
                           0                            0 
               soil_moisture                water_balance 
                           0                            0 
                     SPEI_1M                      SPEI_3M 
                           0                          104 
                     SPEI_6M                      SPEI_9M 
                         260                          416 
                    SPEI_12M                     SPEI_24M 
                         572                         1196 
                      SPI_1M                       SPI_3M 
                           0                          104 
                      SPI_6M                       SPI_9M 
                         260                          416 
                     SPI_12M                      SPI_24M 
                         572                         1196

In [18]:
sapply(data, class)

actual_evapotranspiration                    community 
                   "numeric"                  "character" 
                        date              max_temperature 
                      "Date"                    "numeric" 
             min_temperature potential_evapotranspiration 
                   "numeric"                    "numeric" 
               precipitation                     province 
                   "numeric"                  "character" 
               soil_moisture                water_balance 
                   "numeric"                    "numeric" 
                     SPEI_1M                      SPEI_3M 
                   "numeric"                    "numeric" 
                     SPEI_6M                      SPEI_9M 
                   "numeric"                    "numeric" 
                    SPEI_12M                     SPEI_24M 
                   "numeric"                    "numeric" 
                      SPI_1M                       SPI_3M 
                   "numeric"                    "numeric" 
                      SPI_6M                       SPI_9M 
                   "numeric"                    "numeric" 
                     SPI_12M                      SPI_24M 
                   "numeric"                    "numeric"

In [19]:
output_path = path("..", "data", "processed", "terra_climate")

In [20]:
write_parquet(data, file.path(output_path, "terra_climate_monthly_data.parquet"))